In [ ]:
#hide
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

%load_ext autoreload
%autoreload 2 

In [ ]:
#hide
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")    

# RFPYE
> Este módulo tem como objetivo o processamento e extração otimizada de dados dos arquivos `.bin` de monitoramento do espectro provenientes do script Logger executados nas estações de Monitoramento CRFS RFeye Node. Para tal utilizamos as várias funcionalidades da biblioteca [fastcore](https://fastcore.fast.ai/basics.html), que expande e otimiza as estruturas de dados da linguagem python. 

## Instalação

`Ubuntu`: 

```bash
python -m pip install rfpye
```

`Windows`:

Como parte dessa lib utiliza código c compilado com `Cython`, é preciso que um compilador `C` esteja instalado. Em Windows, uma opção é instalar a versão apropriada do Visual Studio seguindo as orientações do site da Microsoft. No entanto uma solução mais simples e a recomendada é utilizando o `conda`.

Primeiramente instale o [miniconda](https://docs.conda.io/en/latest/miniconda.html). Com o conda instalado e disponível no seu `PATH` ou através do `Anaconda Prompt` execute o comando:

```bash
conda install -c intel libpython m2w64-toolchain -y

echo [build] > %CONDA_PREFIX%\Lib\distutils\distutils.cfg

echo compiler = mingw32 >> %CONDA_PREFIX%\Lib\distutils\distutils.cfg
```

Depois disso basta instalar normalmente a lib:
`python -m pip install rfpye`

Em Linux normalmente o sistema já possui o compilador `gcc` instalado então basta executar o comando `pip install` acima.

## Como utilizar
Abaixo mostramos as funcionalidades principais dos módulos, utilizando-os dentro de algum outro script ou `REPL`

Precisamos necessariamente de um diretório de entrada, contendo um ou mais arquivos `.bin`
> Mude os caminhos abaixo para suas pastas locais

In [ ]:
from fastcore.xtras import Path
from rfpye.utils import get_files
from rich import print

A função abaixo baixa alguns arquivos de exemplo:

In [ ]:
path = Path(r'binfiles')
if not path.exists() or not len(get_files(path, extensions=['.bin'])):
    path = Path('.')
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --output-document 'rfeye002092_210208_T202310_CRFSBINv.5.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --output-document 'rfeye002092_210208_T203131_CRFSBINv.2.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --output-document 'rfeye002292_210208_T202215_CRFSBINv.4.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --output-document 'rfeye002292_210208_T203238_CRFSBINv.3.bin'


A função `parse_bin` é a função principal que encapsula o processamento dos arquivos bin.

In [ ]:
#hide
from nbdev import show_doc

In [ ]:
from rfpye.parser import parse_bin
show_doc(parse_bin)

<h4 id="parse_bin" class="doc_header"><code>parse_bin</code><a href="https://github.com/ronaldokun/rfpye/tree/master/rfpye/parser.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>parse_bin</code>(**`bin_file`**:`Union`\[`str`, `Path`\])

Receives a CRFS binfile and returns a dictionary with the file metadata, a GPS Class and a list with the different Spectrum Classes
A block is a piece of the .bin file with a known start and end and that contains different types of information.
It has several fields: file_type, header, data and footer.
Each field has lengths and information defined in the documentation.
Args:
    bin_file (Union[str, Path]): path to the bin file

Returns:
    Dictionary with the file metadata, file_version, string info, gps and spectrum blocks.

## CRFS Bin - Versão 5
Vamos listar arquivos da última versão do script Logger, Versão 5

In [ ]:
files = get_files(r'D:\OneDrive - ANATEL\Sensores', extensions=['.bin'])
files

(#65) [Path('D:/OneDrive - ANATEL/Sensores/rfeye002073/rfeye002073_210620_T231206.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002080/rfeye002080_691231_T210111.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002081/rfeye002081_210620_T232204.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002083/rfeye002083_210621_T160001.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002084/rfeye002084_210623_T144012.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002086/rfeye002086_210622_T004723.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002087/rfeye002087_210628_T224301.bin'),Path('D:/OneDrive - ANATEL/Sensores/RFeye002090-VCP/rfeye002090-VCP_210623_T024236.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002091/rfeye002091_210618_T145747.bin'),Path('D:/OneDrive - ANATEL/Sensores/rfeye002092/rfeye002092_210603_T205009.bin')...]

In [ ]:
file = files.shuffle()[0]

In [ ]:
%%time
v5 = parse_bin(file)

Wall time: 15.4 s


In [ ]:
print(v5)

{
    'filename': 'rfeye002143_210416_T103920.bin',
    'file_version': 23,
    'string': 'CRFS DATA FILE V023',
    'hostname': 'rfeye002143',
    'method': 'ScriptRFeye2021_v.1',
    'unit_info': 'Stationary',
    'file_number': 0,
    'identifier': 'INFO',
    'gps': GPS Data - Median of Coordinates: -25.51770:-54.55822, Altitude: 247.20 
#Satellites: 11.0 ,
    'spectrum': [SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de
4).', start_mega=50.0, stop_mega=90.0, unit='dBμV/m', ndata=1024, bw=73828, 
processing='peak', antenna_id=0, thresh=-49.5, minimum=-49.5), SpecData(type=67, 
thread_id=110, description='PRD 2021 (Faixa principal 2 de 4).', start_mega=70.0, 
stop_mega=110.0, unit='dBμV/m', ndata=1024, bw=73828, processing='peak', antenna_id=0, 
thresh=-29.5, minimum=-29.5), SpecData(type=67, thread_id=120, description='PRD 2021 (Faixa 
principal 3 de 4).', start_mega=170.0, stop_mega=220.0, unit='dBμV/m', ndata=1280, bw=73828, 
processing='peak', antenna_id=0, thresh=-47.5, minimum=-47.5), SpecData(type=67, 
thread_id=130, description='PRD 2021 (Faixa principal 4 de 4).', start_mega=470.0, 
stop_mega=700.0, unit='dBμV/m', ndata=5888, bw=73828, processing='peak', antenna_id=0, 
thresh=-28.5, minimum=-28.5), SpecData(type=67, thread_id=300, description='PMEC 2021 (Faixa 
1 de 10).', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, bw=18457, 
processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, 
thread_id=310, description='PMEC 2021 (Faixa 2 de 10).', start_mega=155.0, stop_mega=165.0, 
unit='dBm', ndata=1024, bw=18457, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=210, description='PMEF 2021 (Faixa 2 de 6).', 
start_mega=1710.0, stop_mega=1980.0, unit='dBm', ndata=6912, bw=73828, processing='peak', 
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=220, 
description='PMEF 2021 (Faixa 3 de 6).', start_mega=2100.0, stop_mega=1002168.999, 
unit='dBm', ndata=1792, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=230, description='PMEF 2021 (Faixa 4 de 6).', 
start_mega=2290.0, stop_mega=2390.0, unit='dBm', ndata=2560, bw=73828, processing='peak', 
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=240, 
description='PMEF 2021 (Faixa 5 de 6).', start_mega=2500.0, stop_mega=2690.0, unit='dBm', 
ndata=4864, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), 
SpecData(type=67, thread_id=250, description='PMEF 2021 (Faixa 6 de 6).', start_mega=3290.0, 
stop_mega=3700.0, unit='dBm', ndata=10496, bw=73828, processing='peak', antenna_id=0, 
thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=320, description='PMEC 2021 
(Faixa 3 de 10).', start_mega=320.0, stop_mega=340.0, unit='dBm', ndata=512, bw=73828, 
processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, 
thread_id=330, description='PMEC 2021 (Faixa 4 de 10).', start_mega=400.0, stop_mega=410.0, 
unit='dBm', ndata=256, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=340, description='PMEC 2021 (Faixa 5 de 10).', 
start_mega=960.0, stop_mega=1001218.999, unit='dBm', ndata=6656, bw=73828, processing='peak',
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=350, 
description='PMEC 2021 (Faixa 6 de 10).', start_mega=1001388.999, stop_mega=1001428.999, 
unit='dBm', ndata=1280, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=360, description='PMEC 2021 (Faixa 7 de 10).', 
start_mega=1530.0, stop_mega=1001648.999, unit='dBm', ndata=3072, bw=73828, 
processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, 
thread_id=370, description='PMEC 2021 (Faixa 8 de 10).', start_mega=2690.0, 
stop_mega=1002898.999, unit='dBm', ndata=5376, bw=73828, processing='peak',

A saída da função é um dicionário, com os metadados do arquivo:

No entanto as duas chaves mais importantes do dicionário retornado são `gps` e `spectrum`

In [ ]:
from rfpye.utils import getattrs

print(getattrs(v5['gps']))

{'altitude': 247.2, 'latitude': -25.517703, 'longitude': -54.558216, 'num_satellites': 11.0}

Se você imprimir a classe retornada pela chave `gps` é retornado um resumo dos seus atributos:

In [ ]:
print(v5['gps'])

GPS Data - Median of Coordinates: -25.51770:-54.55822, Altitude: 247.20 #Satellites: 11.0

Os atributos listados são os valores consolidados por meio da __mediana__ dos diversos blocos de GPS do arquivo. 

> Caso desejar a lista original de valores, os atributos são os mesmos mas precedidos de `_`, o que os torna __atributos privados__ em python, isso somente quer dizer que não são explicitados em algus métodos como `getattrs`, pois normalmente não são acessíveis diretamente, mas nada impede que sejam acessados.

In [ ]:
v5['gps']._latitude

(#9055) [-25.517706,-25.517675,-25.517693,-25.517695,-25.51768,-25.517693,-25.517711,-25.517717,-25.517697,-25.517709...]

In [ ]:
 v5['gps']._longitude

(#9055) [-54.558221,-54.558219,-54.558237,-54.558247,-54.558232,-54.558226,-54.558242,-54.558246,-54.55823,-54.558237...]

In [ ]:
v5['gps']._altitude

(#9055) [249.4,249.9,247.3,250.6,249.2,244.0,246.3,247.1,242.8,242.9...]

In [ ]:
v5['gps']._num_satellites 

(#9055) [10,10,10,10,10,10,10,11,11,10...]

Cada arquivo bin normalmente possui vários fluxos de espectro distintos, cada fluxo espectral é uma classe Python, na chave `spectrum` é retornado uma lista com todos os fluxos de espectro.

Vamos investigar alguns deles:

In [ ]:
fluxo = v5['spectrum'][0]

Ao imprimir um fluxo é mostrado informações mínimas sobre o seu conteúdo:

In [ ]:
print(fluxo)

Blocks of Type: 67, Thread_id: 100, Start: 50.0MHz. Stop: 90.0MHz

A função `repr` retorna uma representação com todos os metadados do fluxo:

In [ ]:
print(repr(fluxo))

SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de 4).', 
start_mega=50.0, stop_mega=90.0, unit='dBμV/m', ndata=1024, bw=73828, processing='peak', 
antenna_id=0, thresh=-49.5, minimum=-49.5)

No entanto o principal atributo de um fluxo de espectro são os valores de nível medidos, os valores medidos são retornados por meio do atributo: `levels`:

In [ ]:
print(fluxo.levels)

[[26.  22.5 24.  ... 56.5 52.5 43.5]
 [31.5 31.5 29.  ... 54.  55.5 51. ]
 [24.  27.  30.  ... 54.  46.  33. ]
 ...
 [29.  25.  24.5 ... 56.  53.5 45. ]
 [32.5 33.5 32.  ... 56.  50.  36.5]
 [31.  30.5 27.  ... 57.  52.5 40. ]]

In [ ]:
print(f'Formato da matriz com os níveis: {fluxo.levels.shape}')

Formato da matriz com os níveis: (1812, 1024)

O nº de linhas da matriz nos dá o número de pontos medidos naquele dado fluxo e as colunas o número de traços no qual o Span ( Stop - Start ) foi dividido. O número de traços pode ser retornada também por meio da função `len`

In [ ]:
print(len(fluxo))

1812

O atributo anterior retorna uma `numpy.ndarray`, que é um formato eficiente para processamento. 

No entanto temos adicionalmente o método `.matrix()` que retorna a matriz de dados como um _Pandas Dataframe_ formatada com o tempo da medição de cada traço como índice das linhas e as frequências de cada traço como coluna.

Vamos mostrar as cinco primeiras e cinco últimas linhas e colunas. 

In [ ]:
fluxo.matrix().iloc[:5, :5]

Frequencies,50.000000,50.039101,50.078201,50.117302,50.156403
Time,,,,,
2021-04-16 10:40:00.514754,26.0,22.5,24.0,28.5,26.0
2021-04-16 10:45:00.376177,31.5,31.5,29.0,23.5,24.5
2021-04-16 10:50:00.214999,24.0,27.0,30.0,32.5,30.5
2021-04-16 10:55:01.686488,25.0,25.5,29.0,27.0,26.5
2021-04-16 11:00:00.934284,27.5,31.0,30.5,28.0,27.0


In [ ]:
fluxo.matrix().iloc[-5:, -5:]

Frequencies,89.843597,89.882698,89.921799,89.960899,90.000000
Time,,,,,
2021-04-22 17:15:01.577707,53.0,56.5,56.5,54.0,44.0
2021-04-22 17:20:01.742711,55.0,48.5,55.5,55.5,53.0
2021-04-22 17:25:00.873610,47.5,56.5,56.0,53.5,45.0
2021-04-22 17:30:00.641725,50.0,56.0,56.0,50.0,36.5
2021-04-22 17:35:00.463667,48.0,55.5,57.0,52.5,40.0
